In [2]:
## The goal is to clean the master file and transaction file -

import dask.dataframe as dd
import warnings
warnings.filterwarnings("ignore")
dtypes ={'ATS_indicator': 'object',
       'ascii_rptd_vol_tx': 'object',
       'asof_cd': 'object',
       'bsym': 'object',
       'cusip_id': 'object',
       'function': 'object',
       'high_yld_sign_cd': 'object',
       'low_yld_sign_cd': 'object',
       'lsal_yld_sign_cd': 'object',
       'orig_dis_dt': 'object',
       'rptd_pr': 'float64',
       'sale_cndtn2_cd': 'object',
       'sale_cndtn_cd': 'object',
       'spcl_trd_fl': 'object',
       'wis_fl': 'object',
       'yld_sign_cd': 'object',
        'remuneration': 'object'}

trans = dd.read_csv('trans_raw.csv',dtype = dtypes)
trans.head()

/Users/duladula/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


,cusip_id,bond_sym_id,company_symbol,bsym,trd_exctn_dt,trd_exctn_tm,rptd_pr,yld_sign_cd,yld_pt
0,NaN,CHRC4002706,CHRC,NaN,2016-06-28,11:36:45,127.28,NaN,NaN
1,NaN,LBXX.CQ,LBXX,NaN,2016-06-28,11:36:45,122.20,NaN,NaN
2,NaN,LBXX.CQ,LBXX,NaN,2016-06-28,11:36:45,117.20,NaN,NaN
3,NaN,CHRC4376689,CHRC,NaN,2016-06-28,13:15:54,100.00,NaN,NaN
4,NaN,CHRC4376689,CHRC,NaN,2016-06-28,13:15:54,100.00,NaN,NaN


In [3]:
## Drop all the nan bond_sym_id in trans file

import numpy as np
import pandas as pd
trans = trans.dropna(subset = ['bond_sym_id'])
trans = trans.compute()
trans.head()

,cusip_id,bond_sym_id,company_symbol,bsym,trd_exctn_dt,trd_exctn_tm,rptd_pr,yld_sign_cd,yld_pt
0,NaN,CHRC4002706,CHRC,NaN,2016-06-28,11:36:45,127.28,NaN,NaN
1,NaN,LBXX.CQ,LBXX,NaN,2016-06-28,11:36:45,122.20,NaN,NaN
2,NaN,LBXX.CQ,LBXX,NaN,2016-06-28,11:36:45,117.20,NaN,NaN
3,NaN,CHRC4376689,CHRC,NaN,2016-06-28,13:15:54,100.00,NaN,NaN
4,NaN,CHRC4376689,CHRC,NaN,2016-06-28,13:15:54,100.00,NaN,NaN


In [4]:
## change trans date and time to date and time format

trans['trd_exctn_dt'] = pd.to_datetime(trans['trd_exctn_dt'],errors='coerce')
trans['trd_exctn_tm'] = pd.to_datetime(trans['trd_exctn_tm'], format='%H:%M:%S', errors='coerce')

In [5]:
## Drop all the nan time and date

trans = trans.dropna(subset = ['trd_exctn_dt'])
trans = trans.dropna(subset = ['trd_exctn_tm'])

In [6]:
## Group by bond_id, date and sort the time

trans = trans.sort_values(by = ['bond_sym_id', 'trd_exctn_dt', 'trd_exctn_tm'])

In [7]:
## Only keep the last date for each group

trans_clean_lastday= trans.groupby(['bond_sym_id', 'trd_exctn_dt']).last().reset_index()

In [8]:
## drop the time column since we don't use that afterwards

trans_clean = trans_clean_lastday.drop(columns = 'trd_exctn_tm')

In [9]:
trans_clean.shape

(18009984, 8)

In [10]:
trans_clean.head(20)

,bond_sym_id,trd_exctn_dt,cusip_id,company_symbol,bsym,rptd_pr,yld_sign_cd,yld_pt
0,A.GC,2016-06-28,00846UAC5,A,BBG0000KXND3,105.012000,None,2.645854
1,A.GC,2016-06-29,00846UAC5,A,BBG0000KXND3,105.616000,None,2.165466
2,A.GC,2016-06-30,00846UAC5,A,BBG0000KXND3,105.391000,None,2.324178
3,A.GC,2016-07-01,00846UAC5,A,BBG0000KXND3,106.372000,None,1.588240
4,A.GC,2016-07-05,00846UAC5,A,BBG0000KXND3,103.996000,None,3.360572
5,A.GC,2016-07-07,00846UAC5,A,BBG0000KXND3,105.728000,None,2.019925
6,A.GC,2016-07-08,00846UAC5,A,BBG0000KXND3,105.302000,None,2.332121
7,A.GC,2016-07-11,00846UAC5,A,BBG0000KXND3,105.928000,None,1.851203
8,A.GC,2016-07-12,00846UAC5,A,BBG0000KXND3,105.687000,None,2.023519
9,A.GC,2016-07-13,00846UAC5,A,BBG0000KXND3,105.550000,None,2.100214


In [11]:
## count the number of date for each bond, we want to get 1 for each

trans_cleaned1 = trans_clean[['bond_sym_id', 'trd_exctn_dt']].value_counts()
trans_cleaned1

bond_sym_id   trd_exctn_dt
A.GC          2016-06-28      1
MXRQ5111056   2023-08-14      1
              2023-08-30      1
              2023-08-23      1
              2023-08-22      1
                             ..
DVN5328972    2022-11-15      1
              2022-11-16      1
              2022-11-17      1
              2022-11-18      1
ZZTRN5500959  2024-06-28      1
Name: count, Length: 18009984, dtype: int64

In [12]:
# make sure the max is 1
trans_cleaned1.max()

1

In [13]:
## Now, let process some trans date
## check the trading days for each bond

trades_counts = trans_clean.groupby('bond_sym_id')['trd_exctn_dt'].size().reset_index(name = 'daily_count')

trades_counts

,bond_sym_id,daily_count
0,A.GC,158
1,A.GF,626
2,A3900782,818
3,A4020252,1048
4,A4404722,944
...,...,...
177018,ZZCC4900424,4
177019,ZZCC5447693,1
177020,ZZTRN4937481,4
177021,ZZTRN4939132,4


In [14]:
## we only want to keep the bond that the trading days is bigger than 100

trades_counts_100_df = trades_counts[trades_counts['daily_count']>100]['bond_sym_id']
trades_counts_100_df

0                 A.GC
1                 A.GF
2             A3900782
3             A4020252
4             A4404722
              ...     
177000    ZLIOF3945492
177001     ZNGA5008927
177007     ZRLU5196021
177010       ZS5218675
177013      ZTO5647134
Name: bond_sym_id, Length: 27846, dtype: object

In [15]:
master = pd.read_csv('master_raw.csv')
master.head(20)

,cusip_id,sym_cd,COMPANY_SYMBOL,sub_prdct_type,debt_type_cd,cpn_rt,cpn_type_cd,mtrty_dt,dissem,grade
0,000305AA0,AMA.GD,AMA,CORP,CORP,10.250,NaN,2012-10-01,N,NaN
1,000305AA0,AMA.GD,AMA,CORP,CORP,0.000,ZRFX,2012-10-01,N,NaN
2,000305AB8,AMA.GE,AMA,CORP,CORP,10.250,NaN,2012-10-01,N,NaN
3,000305AB8,AMA.GE,AMA,CORP,CORP,10.250,NaN,2012-10-01,Y,NaN
4,000305AB8,AMA.GE,AMA,CORP,CORP,0.000,ZRFX,2012-10-01,Y,NaN
5,000305AB8,AMA.GE,AMA,CORP,CORP,10.250,ZRFX,2012-10-01,Y,NaN
6,000305AB8,AMA.GE,AMA,CORP,CORP,10.250,ZRVR,2012-10-01,Y,NaN
7,000305AB8,AACG3707459,AACG,CORP,CORP,10.250,ZRVR,2012-10-01,Y,NaN
8,000305AC6,AMA.GI,AMA,CORP,CORP,0.000,ZRFX,2012-10-01,N,NaN
9,000305AC6,AMA.GI,AMA,CORP,CORP,10.250,ZRFX,2012-10-01,N,NaN


In [16]:
## filter master bond

master = master[(master['mtrty_dt']>='2016-06-28')& (master['sub_prdct_type']=='CORP')&(master['debt_type_cd']=='CORP')]
master = master[master['sym_cd'].isin(trades_counts_100_df)]
master.head(20)

,cusip_id,sym_cd,COMPANY_SYMBOL,sub_prdct_type,debt_type_cd,cpn_rt,cpn_type_cd,mtrty_dt,dissem,grade
86,00037BAA0,ABB3852123,ABB,CORP,CORP,1.625,FXPV,2017-05-08,Y,NaN
88,00037BAB8,ABB3852125,ABB,CORP,CORP,2.875,FXPV,2022-05-08,Y,NaN
91,00037BAC6,ABB3852142,ABB,CORP,CORP,4.375,FXPV,2042-05-08,Y,NaN
94,00037BAD4,ABB4615652,ABB,CORP,CORP,2.800,FXPV,2020-04-03,Y,NaN
97,00037BAE2,ABB4615653,ABB,CORP,CORP,3.375,FXPV,2023-04-03,Y,NaN
100,00037BAF9,ABB4615654,ABB,CORP,CORP,3.800,FXPV,2028-04-03,Y,NaN
573,00080QAB1,ABN3705326,ABN,CORP,CORP,4.650,FXPV,2018-06-04,Y,NaN
627,00081TAH1,ACCO4020559,ACCO,CORP,CORP,6.750,FXPV,2020-04-30,Y,NaN
1724,00101JAF3,ADT3991008,ADT,CORP,CORP,3.500,FXPV,2022-07-15,Y,NaN
1727,00101JAG1,ADT3991007,ADT,CORP,CORP,4.875,FXPV,2042-07-15,Y,NaN


In [17]:
master_counts = master.groupby('sym_cd').size().reset_index(name = 'bond_count')
master_counts

,sym_cd,bond_count
0,A.GC,2
1,A.GF,1
2,A3900782,1
3,A4020252,1
4,A4404722,1
...,...,...
19664,ZION4135554,1
19665,ZION4669811,1
19666,ZION4804244,1
19667,ZION4902922,1


In [18]:
## Only reserve bond_count with no duplicated records here

master_duplicate_df = master_counts[master_counts['bond_count']==1]['sym_cd']

In [19]:
master_cleaned = master[master['sym_cd'].isin(master_duplicate_df)]
master_cleaned.head(20)

,cusip_id,sym_cd,COMPANY_SYMBOL,sub_prdct_type,debt_type_cd,cpn_rt,cpn_type_cd,mtrty_dt,dissem,grade
86,00037BAA0,ABB3852123,ABB,CORP,CORP,1.625,FXPV,2017-05-08,Y,NaN
88,00037BAB8,ABB3852125,ABB,CORP,CORP,2.875,FXPV,2022-05-08,Y,NaN
91,00037BAC6,ABB3852142,ABB,CORP,CORP,4.375,FXPV,2042-05-08,Y,NaN
94,00037BAD4,ABB4615652,ABB,CORP,CORP,2.800,FXPV,2020-04-03,Y,NaN
97,00037BAE2,ABB4615653,ABB,CORP,CORP,3.375,FXPV,2023-04-03,Y,NaN
100,00037BAF9,ABB4615654,ABB,CORP,CORP,3.800,FXPV,2028-04-03,Y,NaN
573,00080QAB1,ABN3705326,ABN,CORP,CORP,4.650,FXPV,2018-06-04,Y,NaN
627,00081TAH1,ACCO4020559,ACCO,CORP,CORP,6.750,FXPV,2020-04-30,Y,NaN
1724,00101JAF3,ADT3991008,ADT,CORP,CORP,3.500,FXPV,2022-07-15,Y,NaN
1727,00101JAG1,ADT3991007,ADT,CORP,CORP,4.875,FXPV,2042-07-15,Y,NaN


In [20]:
master_cleaned.shape

(16504, 10)

In [21]:
trans_clean_final = trans_clean[trans_clean['bond_sym_id'].isin(master_cleaned['sym_cd'])]

In [22]:
trans_clean_final.shape

(12194572, 8)

In [23]:
## Set index for master 
master_cleaned.set_index('sym_cd', inplace=True)

In [30]:
## Set index for trans 

trans_clean_final['bond_id'] = trans_clean_final['bond_sym_id'].astype(str) + '_' + trans_clean_final['trd_exctn_dt'].astype(str)

# Set the new column as the index

trans_clean_final.set_index('bond_id', inplace=True)

In [31]:
trans_clean_final.to_csv('trans_clean.csv')

In [32]:
master_cleaned.to_csv('master_clean.csv')